Camera calibration


In [6]:
# Import required modules 
import cv2 
import numpy as np 
import os 
import glob 


# Define the dimensions of checkerboard 
CHECKERBOARD = (6, 9) 


# stop the iteration when specified 
# accuracy, epsilon, is reached or 
# specified number of iterations are completed. 
criteria = (cv2.TERM_CRITERIA_EPS +
			cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) 


# Vector for 3D points 
threedpoints = [] 

# Vector for 2D points 
twodpoints = [] 


# 3D points real world coordinates 
objectp3d = np.zeros((1, CHECKERBOARD[0] 
					* CHECKERBOARD[1], 
					3), np.float32) 
objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 
							0:CHECKERBOARD[1]].T.reshape(-1, 2) 
prev_img_shape = None


# Extracting path of individual image stored 
# in a given directory. Since no path is 
# specified, it will take current directory 
# jpg files alone 
#images = glob.glob('./images/*.jpg')
#images = glob.glob('cc1.jpg') 

images = ['cc1.jpg','cc2.jpg','cc3.jpg','cc4.jpg','cc5.jpg']


for filename in images: 
	image = cv2.imread(filename) 
	grayColor = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

	# Find the chess board corners 
	# If desired number of corners are 
	# found in the image then ret = true 
	ret, corners = cv2.findChessboardCorners( 
					grayColor, CHECKERBOARD, 
					cv2.CALIB_CB_ADAPTIVE_THRESH 
					+ cv2.CALIB_CB_FAST_CHECK +
					cv2.CALIB_CB_NORMALIZE_IMAGE) 

	# If desired number of corners can be detected then, 
	# refine the pixel coordinates and display 
	# them on the images of checker board 
	if ret == True: 
		threedpoints.append(objectp3d) 

		# Refining pixel coordinates 
		# for given 2d points. 
		corners2 = cv2.cornerSubPix( 
			grayColor, corners, (11, 11), (-1, -1), criteria) 

		twodpoints.append(corners2) 

		# Draw and display the corners 
		image = cv2.drawChessboardCorners(image, 
										CHECKERBOARD, 
										corners2, ret) 

	cv2.imshow('img', image) 
	cv2.waitKey(0) 

cv2.destroyAllWindows() 

h, w = image.shape[:2] 


# Perform camera calibration by 
# passing the value of above found out 3D points (threedpoints) 
# and its corresponding pixel coordinates of the 
# detected corners (twodpoints) 
ret, matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera( 
	threedpoints, twodpoints, grayColor.shape[::-1], None, None) 


# Displaying required output 
print(" Camera matrix:") 
print(matrix) 

print("\n Distortion coefficient:") 
print(distortion) 

print("\n Rotation Vectors:") 
print(r_vecs) 

print("\n Translation Vectors:") 
print(t_vecs) 


 Camera matrix:
[[990.29650125   0.         654.29114674]
 [  0.         986.90530304 379.90130831]
 [  0.           0.           1.        ]]

 Distortion coefficient:
[[-0.18963786  0.44927928  0.00393996 -0.00135861 -0.80596688]]

 Rotation Vectors:
(array([[-0.13468437],
       [ 0.08544687],
       [-0.02183672]]), array([[-0.53383558],
       [ 0.47088315],
       [ 1.53518793]]), array([[-0.14131138],
       [-0.28351896],
       [-0.23074169]]), array([[-0.6668972 ],
       [-0.05847564],
       [ 0.01130626]]), array([[ 0.15950927],
       [ 0.04887307],
       [-0.05826589]]))

 Translation Vectors:
(array([[-2.26076984],
       [-2.34440557],
       [30.85009885]]), array([[ 4.50910171],
       [ 0.03755518],
       [25.18322117]]), array([[-3.04388852],
       [-1.53596068],
       [31.74039604]]), array([[ 0.46149125],
       [-1.52405664],
       [30.1278694 ]]), array([[-0.53037716],
       [-4.23876978],
       [31.35002414]]))


In [7]:
# Save the camera calibration result
calibration_file = "camera_calibration.npz"

# Saving the calibration results to a file
np.savez(calibration_file, camera_matrix=matrix, distortion_coefficient=distortion, rotation_vectors=r_vecs, translation_vectors=t_vecs)

print(f"\nCalibration data saved to {calibration_file}")


Calibration data saved to camera_calibration.npz


In [8]:
#load the camera calibration data
data = np.load('camera_calibration.npz')
camera_matrix = data['camera_matrix']
distortion_coefficient = data['distortion_coefficient']
rotation_vectors = data['rotation_vectors']
translation_vectors = data['translation_vectors']
